In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import random

In [3]:
data = pd.read_csv('Data\LeagueofLegends.csv')
champ = pd.read_csv('Data\Champions.csv')
team = pd.read_csv('Data\Team.csv')
testdatas = pd.read_csv('Data\Test.csv')
data.head()

,blueTeamTag,redTeamTag,gamelength,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp,Result
0,3,12,30,68,56,112,8,4,103,85,19,48,108,0
1,12,3,43,34,25,121,70,110,68,56,59,63,4,1
2,1,8,40,62,85,59,48,11,29,25,65,63,93,0
3,8,1,39,32,25,51,63,14,34,85,2,70,110,1
4,9,6,45,103,85,65,70,11,64,22,59,109,4,1


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 535 entries, 0 to 534
Data columns (total 14 columns):
blueTeamTag         535 non-null int64
redTeamTag          535 non-null int64
gamelength          535 non-null int64
blueTopChamp        535 non-null int64
blueJungleChamp     535 non-null int64
blueMiddleChamp     535 non-null int64
blueADCChamp        535 non-null int64
blueSupportChamp    535 non-null int64
redTopChamp         535 non-null int64
redJungleChamp      535 non-null int64
redMiddleChamp      535 non-null int64
redADCChamp         535 non-null int64
redSupportChamp     535 non-null int64
Result              535 non-null int64
dtypes: int64(14)
memory usage: 58.6 KB


In [45]:
data.shape

(535, 14)

In [46]:
labels = data['Result']
tY = testdatas['Result']

print(data.head())
print(labels.shape)

   blueTeamTag  redTeamTag  gamelength  blueTopChamp  blueJungleChamp  \
0            3          12          30            68               56   
1           12           3          43            34               25   
2            1           8          40            62               85   
3            8           1          39            32               25   
4            9           6          45           103               85   

   blueMiddleChamp  blueADCChamp  blueSupportChamp  redTopChamp  \
0              112             8                 4          103   
1              121            70               110           68   
2               59            48                11           29   
3               51            63                14           34   
4               65            70                11           64   

   redJungleChamp  redMiddleChamp  redADCChamp  redSupportChamp  Result  
0              85              19           48              108       0  
1         

In [47]:
features = data.drop(['Result'], axis = 1)

tX = testdatas.drop(['Result'], axis = 1)
features.head()

,blueTeamTag,redTeamTag,gamelength,blueTopChamp,blueJungleChamp,blueMiddleChamp,blueADCChamp,blueSupportChamp,redTopChamp,redJungleChamp,redMiddleChamp,redADCChamp,redSupportChamp
0,3,12,30,68,56,112,8,4,103,85,19,48,108
1,12,3,43,34,25,121,70,110,68,56,59,63,4
2,1,8,40,62,85,59,48,11,29,25,65,63,93
3,8,1,39,32,25,51,63,14,34,85,2,70,110
4,9,6,45,103,85,65,70,11,64,22,59,109,4


In [48]:
features, labels = np.array(features), np.array(labels)
teams = np.array(team)
champs = np.array(champ)
tX = np.array(tX)
tY = np.array(tY)
print(len(features), len(labels))

535 535


In [49]:
split_frac = 1.0
n_records = len(features)
t_n_records = len(tX)
split_idx = int(split_frac*n_records)
t_split_idx = int(t_n_records)

In [50]:
train_X, train_Y = features[:split_idx], labels[:split_idx]
test_X, test_Y = tX[:t_split_idx], tY[:t_split_idx]

In [51]:
n_labels= 2
n_features = 13

In [52]:
learning_rate = 0.001
n_epochs= 300
hidden_output = 50

In [77]:
def nn_model():
    
    tf.reset_default_graph()

    inputs = tf.placeholder(tf.float32,[None, n_features], name ='inputs' )
    labels = tf.placeholder(tf.int32, [None,], name='output')
    labels_one_hot = tf.one_hot(labels, n_labels)

    W1 = tf.Variable(tf.truncated_normal([n_features,hidden_output], stddev=0.1))
        
    b1 = tf.Variable(tf.zeros([hidden_output]))
    L1 = tf.nn.relu(tf.matmul(inputs, W1) + b1)

    W2 = tf.Variable(tf.truncated_normal([hidden_output,hidden_output], stddev=0.1))
    b2 = tf.Variable(tf.zeros([hidden_output]))
    L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

    W3 = tf.Variable(tf.truncated_normal([hidden_output,hidden_output], stddev=0.1))
    b3 = tf.Variable(tf.zeros([hidden_output]))
    L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
    
    W4 = tf.Variable(tf.truncated_normal([hidden_output,hidden_output], stddev=0.1))
    b4 = tf.Variable(tf.zeros([hidden_output]))
    L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
    
    W5 = tf.Variable(tf.truncated_normal([hidden_output, n_labels], stddev=0.1))
    b5 = tf.Variable(tf.zeros(n_labels))
    
    logits = tf.nn.bias_add(tf.matmul(L4, W5), b5)
    hypothesis = tf.nn.softmax(logits)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_one_hot))
    
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        
        sess.run(init)
        file_writer = tf.summary.FileWriter('./logs/1', sess.graph)
        file_writer.add_graph(sess.graph)
        for epoch in range(n_epochs):
            feed = {inputs: train_X, labels: train_Y}
            _, loss = sess.run([optimizer, cost], feed_dict=feed)
           
            print('Epoch:', '%04d' % (epoch + 1),
            'cost =', '{:.9f}'.format(loss))
            
        print('Learning Finished!')
    
        
        correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(labels_one_hot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("Accuracy:", accuracy.eval(feed))
        
        r = random.randint(0, n_records-1)
        print("\nTest1")
        
        print("\nBLUE : ", teams[train_X[r][0]-1][1],"(",champs[train_X[r][3]-1][1], ", ",champs[train_X[r][4]-1][1], ", ",champs[train_X[r][5]-1][1], ", ",champs[train_X[r][6]-1][1], ", ",champs[train_X[r][7]-1][1], ")", " vs RED : ", teams[train_X[r][1]-1][1],"(",champs[train_X[r][8]-1][1], ", ",champs[train_X[r][9]-1][1], ", ",champs[train_X[r][10]-1][1], ", ",champs[train_X[r][11]-1][1], ", ",champs[train_X[12][7]-1][1], ")")
        if train_Y[r] == 0:
            print("\nActual Value : red team win!")
        else:
            print("\nActual Value : blue team win!")
        if sess.run(tf.argmax(hypothesis, 1), feed_dict={inputs: train_X[r:r+1]}) == [0]:
            print("Prediction : red team win!")
        else:
            print("Prediction : blue team win!")
        
        r2 = random.randint(0, t_n_records - 1)
        print("\nTest2")
        
        
        print("\nBLUE : ", teams[test_X[r2][0]-1][1],"(",champs[test_X[r2][3]-1][1], ", ",champs[test_X[r2][4]-1][1], ", ",champs[test_X[r2][5]-1][1], ", ",champs[test_X[r2][6]-1][1], ", ",champs[test_X[r2][7]-1][1], ")", " vs RED : ", teams[test_X[r2][1]-1][1],"(",champs[test_X[r2][8]-1][1], ", ",champs[test_X[r2][9]-1][1], ", ",champs[test_X[r2][10]-1][1], ", ",champs[test_X[r2][11]-1][1], ", ",champs[test_X[r2][12]-1][1], ")")
        if test_Y[r2] == 0:
            print("\nActual Value : red team win!")
        else:
            print("\nActual Value : blue team win!")
        if sess.run(tf.argmax(hypothesis, 1), feed_dict={inputs: test_X[r2:r2+1]}) == [0]:
            print("Prediction : red team win!")
        else:
            print("Prediction : blue team win!")
        
        correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(labels_one_hot, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("Accuracy:", accuracy.eval({inputs: test_X, labels: test_Y}))
        
        print("\nTest3")
        for i in range(0,8):
            print("\nBLUE : ", teams[test_X[i][0]-1][1],"(",champs[test_X[r2][3]-1][1], ", ",champs[test_X[i][4]-1][1], ", ",champs[test_X[i][5]-1][1], ", ",champs[test_X[i][6]-1][1], ", ",champs[test_X[i][7]-1][1], ")", " vs RED : ", teams[test_X[i][1]-1][1],"(",champs[test_X[i][8]-1][1], ", ",champs[test_X[i][9]-1][1], ", ",champs[test_X[i][10]-1][1], ", ",champs[test_X[i][11]-1][1], ", ",champs[test_X[i][12]-1][1], ")")
            if sess.run(tf.argmax(hypothesis, 1), feed_dict={inputs: test_X[i:i+1]}) == [0]:
                print("Prediction : red team win!")
            else:
                print("Prediction : blue team win!")

In [79]:
nn_model()

Epoch: 0001 cost = 0.710663021
Epoch: 0002 cost = 0.705190659
Epoch: 0003 cost = 0.690780222
Epoch: 0004 cost = 0.680447996
Epoch: 0005 cost = 0.678679883
Epoch: 0006 cost = 0.674389184
Epoch: 0007 cost = 0.669621527
Epoch: 0008 cost = 0.668947041
Epoch: 0009 cost = 0.666690469
Epoch: 0010 cost = 0.662883639
Epoch: 0011 cost = 0.660844445
Epoch: 0012 cost = 0.659735978
Epoch: 0013 cost = 0.656959951
Epoch: 0014 cost = 0.653674543
Epoch: 0015 cost = 0.651179790
Epoch: 0016 cost = 0.648944199
Epoch: 0017 cost = 0.645773470
Epoch: 0018 cost = 0.642575145
Epoch: 0019 cost = 0.640116155
Epoch: 0020 cost = 0.637370229
Epoch: 0021 cost = 0.633728266
Epoch: 0022 cost = 0.630951285
Epoch: 0023 cost = 0.628498912
Epoch: 0024 cost = 0.625278294
Epoch: 0025 cost = 0.622290492
Epoch: 0026 cost = 0.619178355
Epoch: 0027 cost = 0.615475416
Epoch: 0028 cost = 0.612002611
Epoch: 0029 cost = 0.608410835
Epoch: 0030 cost = 0.604359210
Epoch: 0031 cost = 0.600685596
Epoch: 0032 cost = 0.596632659
Epoch: 0

Epoch: 0294 cost = 0.002807007
Epoch: 0295 cost = 0.002775753
Epoch: 0296 cost = 0.002743949
Epoch: 0297 cost = 0.002714303
Epoch: 0298 cost = 0.002683756
Epoch: 0299 cost = 0.002653796
Epoch: 0300 cost = 0.002626271
Learning Finished!
Accuracy: 1.0

Test1

BLUE :  CJ ( Gangplank ,  Rek'Sai ,  Twisted Fate ,  Jhin ,  Alistar )  vs RED :  kt ( Gnar ,  Gragas ,  Zilean ,  Ashe ,  Alistar )

Actual Value : red team win!
Prediction : red team win!

Test2

BLUE :  JAG ( Rumble ,  Rek'Sai ,  Galio ,  Kalista ,  Braum )  vs RED :  SKT ( Jarvan IV ,  LeeSin ,  Cassiopeia ,  Ashe ,  Tahm Kench )

Actual Value : red team win!
Prediction : red team win!
Accuracy: 0.685185

Test3

BLUE :  LZ ( Rumble ,  Sejuani ,  Taliyah ,  Kalista ,  Taric )  vs RED :  SSG ( Gnar ,  Gragas ,  Orianna ,  Varus ,  Braum )
Prediction : blue team win!

BLUE :  JAG ( Rumble ,  Rek'Sai ,  Corki ,  Kalista ,  Thresh )  vs RED :  AFs ( Trundle ,  Gragas ,  Cassiopeia ,  Ashe ,  Zyra )
Prediction : red team win!

BLUE : 